In [ ]:

from tensorflow.keras.layers import Input, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1
from tensorflow.keras.layers import Activation
import time
import os
os.environ['XILINX_VITIS'] = '/tools/Xilinx/Vitis/2024.2'
os.environ['PATH'] = '/tools/Xilinx/Vivado/2020.1/bin:' + os.environ['PATH']
os.environ['PATH'] = '/tools/Xilinx/Vitis_HLS/2024.2/bin:' + os.environ['PATH']
from model_utils import save_model


2025-04-08 00:10:37.719712: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-08 00:10:37.778484: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-08 00:10:37.780413: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-04-08 00:10:37.780419: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to [0, 1]
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# Reshape to add channel dimension (28x28x1)
x_train = x_train.reshape((-1, 28, 28, 1))
x_test  = x_test.reshape((-1, 28, 28, 1))

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

# Split off a validation set
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

# Create tf.data.Dataset objects (optional but recommended for performance)
batch_size = 1024

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
val_data   = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)
test_data  = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Optional: set number of epochs
n_epochs = 10

input_shape = (28, 28, 1)
n_classes = 10

2025-04-08 00:10:38.805005: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-04-08 00:10:38.805021: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-04-08 00:10:38.805031: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (theodoros-MS-7D75): /proc/driver/nvidia/version does not exist
2025-04-08 00:10:38.805218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization

filters_per_conv_layer = [16, 16, 24]
neurons_per_dense_layer = [42, 64]

x = x_in = Input(input_shape)

for i, f in enumerate(filters_per_conv_layer):
    print(('Adding convolutional block {} with N={} filters').format(i, f))
    x = Conv2D(
        int(f),
        kernel_size=(3, 3),
        strides=(1, 1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
        use_bias=False,
        name='conv_{}'.format(i),
    )(x)
    x = BatchNormalization(name='bn_conv_{}'.format(i))(x)
    x = Activation('relu', name='conv_act_%i' % i)(x)
    x = MaxPooling2D(pool_size=(2, 2), name='pool_{}'.format(i))(x)
x = Flatten()(x)

for i, n in enumerate(neurons_per_dense_layer):
    print(('Adding dense block {} with N={} neurons').format(i, n))
    x = Dense(n, kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), name='dense_%i' % i, use_bias=False)(x)
    x = BatchNormalization(name='bn_dense_{}'.format(i))(x)
    x = Activation('relu', name='dense_act_%i' % i)(x)
x = Dense(int(n_classes), name='output_dense')(x)
x_out = Activation('softmax', name='output_softmax')(x)

baseline_model = Model(inputs=[x_in], outputs=[x_out], name='keras_baseline')

LOSS = tf.keras.losses.CategoricalCrossentropy()
OPTIMIZER = tf.keras.optimizers.legacy.Adam(learning_rate=3e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)

baseline_model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=["accuracy"])

# Copy the model to a new variable
cloned_model = tf.keras.models.clone_model(baseline_model)
cloned_model = tf.keras.models.clone_model(baseline_model)


Adding convolutional block 0 with N=16 filters
Adding convolutional block 1 with N=16 filters
Adding convolutional block 2 with N=24 filters
Adding dense block 0 with N=42 neurons
Adding dense block 1 with N=64 neurons


In [4]:
from qkeras.autoqkeras import *
from qkeras import *

# Compile the cloned model
cloned_model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=["accuracy"])
# Train the cloned model
cloned_model.fit(train_data, epochs=n_epochs, validation_data=val_data, verbose=2)

print_qmodel_summary(cloned_model)

# Evaluate the cloned model
loss, accuracy = cloned_model.evaluate(test_data, verbose=2)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")
# Save the model
model_dir = "models"
model_name = "keras_baseline"
model_path = os.path.join(model_dir, model_name)
save_model(cloned_model, model_path)

Epoch 1/10
53/53 - 2s - loss: 0.7902 - accuracy: 0.8021 - val_loss: 1.0894 - val_accuracy: 0.7868 - 2s/epoch - 30ms/step
Epoch 2/10
53/53 - 1s - loss: 0.2408 - accuracy: 0.9576 - val_loss: 1.4653 - val_accuracy: 0.5740 - 1s/epoch - 20ms/step
Epoch 3/10
53/53 - 1s - loss: 0.1882 - accuracy: 0.9718 - val_loss: 1.6147 - val_accuracy: 0.5325 - 1s/epoch - 20ms/step
Epoch 4/10
53/53 - 1s - loss: 0.1633 - accuracy: 0.9783 - val_loss: 1.1523 - val_accuracy: 0.6440 - 1s/epoch - 20ms/step
Epoch 5/10
53/53 - 1s - loss: 0.1458 - accuracy: 0.9819 - val_loss: 0.9740 - val_accuracy: 0.7042 - 1s/epoch - 19ms/step
Epoch 6/10
53/53 - 1s - loss: 0.1339 - accuracy: 0.9848 - val_loss: 0.4856 - val_accuracy: 0.8608 - 1s/epoch - 19ms/step
Epoch 7/10
53/53 - 1s - loss: 0.1251 - accuracy: 0.9865 - val_loss: 0.3726 - val_accuracy: 0.9080 - 1s/epoch - 19ms/step
Epoch 8/10
53/53 - 1s - loss: 0.1152 - accuracy: 0.9878 - val_loss: 0.2097 - val_accuracy: 0.9585 - 1s/epoch - 19ms/step
Epoch 9/10
53/53 - 1s - loss: 0.

In [5]:
from netron_embed import view_model

# View your model inline (any format: .onnx, .h5, .pb, etc.)
view_model(model_path+".h5")

Serving 'models/keras_baseline.h5' at http://localhost:42315


In [6]:
from qkeras import print_qstats

# for automatic quantization
import pprint
from qkeras.autoqkeras import *
from qkeras import *
from qkeras.utils import model_quantize

from qkeras.qtools import run_qtools
from qkeras.qtools import settings as qtools_settings
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from qkeras import quantized_bits
from qkeras import QDense, QActivation

q = run_qtools.QTools(
    baseline_model,
    process="horowitz",
    source_quantizers=[quantized_bits(16, 5, 1)],
    is_inference=True,
    weights_path=None,
    keras_quantizer="fp16",
    keras_accumulator="fp16",
    for_reference=False,
)
q.qtools_stats_print()

energy_dict = q.pe(
    weights_on_memory="fixed", activations_on_memory="fixed", min_sram_size=8 * 16 * 1024 * 1024, rd_wr_on_io=False
)

# get stats of energy distribution in each layer
energy_profile = q.extract_energy_profile(qtools_settings.cfg.include_energy, energy_dict)
# extract sum of energy of each layer according to the rule specified in
# qtools_settings.cfg.include_energy
total_energy = q.extract_energy_sum(qtools_settings.cfg.include_energy, energy_dict)

pprint.pprint(energy_profile)
print()

print("Total energy: {:.6f} uJ".format(total_energy / 1000000.0))

Instructions for updating:
Use ref() instead.
{
    "source_quantizers": [
        {
            "quantizer_type": "quantized_bits",
            "bits": 16,
            "int_bits": 6,
            "is_signed": true
        }
    ],
    "conv_0": {
        "layer_type": "Conv2D",
        "input_quantizer_list": [
            {
                "quantizer_type": "quantized_bits",
                "bits": 16,
                "int_bits": 6,
                "is_signed": true
            }
        ],
        "weight_quantizer": {
            "quantizer_type": "floating_point",
            "bits": 16,
            "shape": [
                3,
                3,
                1,
                16
            ]
        },
        "multiplier": {
            "quantizer_type": "floating_point",
            "bits": 16,
            "op_type": "mul"
        },
        "accumulator": {
            "quantizer_type": "floating_point",
            "bits": 16,
            "op_type": "add"
        },
    

In [13]:
# These are the quantizers we'll test in the bayesian optimization
quantization_config = {
    "kernel": {
        "quantized_bits(2,0,1,alpha=1.0)": 2,
        "quantized_bits(4,0,1,alpha=1.0)": 4,
        "quantized_bits(6,0,1,alpha=1.0)": 6,
        "quantized_bits(8,0,1,alpha=1.0)": 8,
    },
    "bias": {
        "quantized_bits(2,0,1,alpha=1.0)": 2,
        "quantized_bits(4,0,1,alpha=1.0)": 4,
        "quantized_bits(6,0,1,alpha=1.0)": 6,
        "quantized_bits(8,0,1,alpha=1.0)": 8,
    },
    "activation": {
        "quantized_relu(3,1)": 3,
        "quantized_relu(4,2)": 4,
        "quantized_relu(8,2)": 8,
        "quantized_relu(8,4)": 8,
        "quantized_relu(16,6)": 16,
    },
    "linear": {
        "quantized_bits(2,0,1,alpha=1.0)": 2,
        "quantized_bits(4,0,1,alpha=1.0)": 4,
        "quantized_bits(6,0,1,alpha=1.0)": 6,
        "quantized_bits(8,0,1,alpha=1.0)": 8,
    },
}


# These are the layer types we will quantize
limit = {
    "conv": [8, 8, 16],
    "dense": [8, 8, 16],
    "act": [16]
}


# Use this if you want to minimize the model bit size
goal_bits = {
    "type": "bits",
    "params": {
        "delta_p": 2.0,  # We tolerate up to a +8% accuracy change
        "delta_n": 2.0,  # We tolerate down to a -5% accuracy change
        "rate": 2.0,  # We want a x2 times smaller model
        "stress": 1.0,  # Force the reference model size to be smaller by setting stress<1
        "input_bits": 8,
        "output_bits": 8,
        "ref_bits": 8,
        "config": {"default": ["parameters", "activations"]},
    },
}

# Use this if you want to minimize the model energy consumption
goal_energy = {
    "type": "energy",
    "params": {
        "delta_p": 8.0,
        "delta_n": 8.0,
        "rate": 2.0,
        "stress": 1.0,
        "process": "horowitz",
        "parameters_on_memory": ["sram", "sram"],
        "activations_on_memory": ["sram", "sram"],
        "rd_wr_on_io": [False, False],
        "min_sram_size": [0, 0],
        "source_quantizers": ["fp32"],
        "reference_internal": "int8",
        "reference_accumulator": "int32",
    },
}

run_config = {
    "goal": goal_bits,
    "quantization_config": quantization_config,
    "learning_rate_optimizer": False,
    "transfer_weights": False,  # Randomely initialize weights
    "mode": "bayesian",  # This can be bayesian,random,hyperband
    "seed": 42,
    "limit": limit,
    #"tune_filters": "layer",
    #"tune_filters_exceptions": "^output",
    "tune_filters": "none",
    "tune_filters_exceptions": "",  
    "distribution_strategy": None,
    "max_trials": 5,  # Let's just do 5 trials for this demonstrator, ideally you should do as many as possible
}

In [14]:
from qkeras.autoqkeras import AutoQKeras
autoqk = AutoQKeras(
    model=baseline_model,
    output_dir="autoqk_results",
    **run_config
)

Limit configuration:{"conv": [8, 8, 16], "dense": [8, 8, 16], "act": [16]}
learning_rate: 0.003000000026077032
Model: "keras_baseline"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv_0 (QConv2D)            (None, 26, 26, 16)        144       
                                                                 
 bn_conv_0 (BatchNormalizati  (None, 26, 26, 16)       64        
 on)                                                             
                                                                 
 conv_act_0 (QActivation)    (None, 26, 26, 16)        0         
                                                                 
 pool_0 (MaxPooling2D)       (None, 13, 13, 16)        0         
                                                                 
 conv_1

In [15]:


space = autoqk.tuner.oracle.get_space()
print("\n🔍 Registered hyperparameters:")
for hp in space.space:
    print(f"• {hp.name}: {hp.values}")


autoqk.fit(
    x=train_data,
    validation_data=val_data,
    epochs=15  # Or however many you want for each trial
)

Trial 5 Complete [00h 00m 44s]
val_score: 0.9914345741271973

Best val_score So Far: 0.9914345741271973
Total elapsed time: 00h 02m 12s


In [16]:
aqmodel = autoqk.get_best_model()
print_qmodel_summary(aqmodel)

# Train for the full epochs
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
]

learning_rate: 0.003000000026077032
Model: "keras_baseline"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv_0 (QConv2D)            (None, 26, 26, 16)        144       
                                                                 
 bn_conv_0 (BatchNormalizati  (None, 26, 26, 16)       64        
 on)                                                             
                                                                 
 conv_act_0 (QActivation)    (None, 26, 26, 16)        0         
                                                                 
 pool_0 (MaxPooling2D)       (None, 13, 13, 16)        0         
                                                                 
 conv_1 (QConv2D)            (None, 11, 11, 16)        2304      
                

In [17]:
start = time.time()
history = aqmodel.fit(train_data, epochs=n_epochs, validation_data=val_data, callbacks=callbacks, verbose=1)
end = time.time()
print('\n It took {} minutes to train!\n'.format((end - start) / 60.0))

Epoch 1/10
53/53 [==============================] - 4s 57ms/step - loss: 0.5604 - acc: 0.8708 - trial: 109680.0000 - score: 0.8853 - val_loss: 1.7038 - val_acc: 0.6353 - val_trial: 109680.0000 - val_score: 0.6459 - lr: 0.0030
Epoch 2/10
53/53 [==============================] - 3s 54ms/step - loss: 0.2421 - acc: 0.9673 - trial: 109680.0000 - score: 0.9835 - val_loss: 1.0761 - val_acc: 0.6888 - val_trial: 109680.0000 - val_score: 0.7003 - lr: 0.0030
Epoch 3/10
53/53 [==============================] - 3s 54ms/step - loss: 0.2017 - acc: 0.9753 - trial: 109680.0000 - score: 0.9916 - val_loss: 1.8768 - val_acc: 0.5097 - val_trial: 109680.0000 - val_score: 0.5182 - lr: 0.0030
Epoch 4/10
53/53 [==============================] - 3s 55ms/step - loss: 0.1837 - acc: 0.9777 - trial: 109680.0000 - score: 0.9941 - val_loss: 1.2711 - val_acc: 0.6157 - val_trial: 109680.0000 - val_score: 0.6259 - lr: 0.0030
Epoch 5/10
53/53 [==============================] - 3s 55ms/step - loss: 0.1675 - acc: 0.9803 - 

In [18]:
# This model has some remnants from the optimization procedure attached to it, so let's define a new one
aqmodel.save_weights("autoqkeras_cnn_weights.h5")

layers = [l for l in aqmodel.layers]
x = layers[0].output
for i in range(1, len(layers)):
    x = layers[i](x)

new_model = Model(inputs=[layers[0].input], outputs=[x])
LOSS = tf.keras.losses.CategoricalCrossentropy()
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=3e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)

new_model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=["accuracy"])
new_model.summary()
new_model.load_weights("autoqkeras_cnn_weights.h5")

print_qmodel_summary(new_model)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv_0 (QConv2D)            (None, 26, 26, 16)        144       
                                                                 
 bn_conv_0 (BatchNormalizati  (None, 26, 26, 16)       64        
 on)                                                             
                                                                 
 conv_act_0 (QActivation)    (None, 26, 26, 16)        0         
                                                                 
 pool_0 (MaxPooling2D)       (None, 13, 13, 16)        0         
                                                                 
 conv_1 (QConv2D)            (None, 11, 11, 16)        2304      
                                                             

In [19]:
# Save the model 
new_model_path = os.path.join(model_dir, "new_model")
save_model(new_model, new_model_path)

view_model(new_model_path+".h5")

✅ Model saved to: models/new_model.h5
Serving 'models/new_model.h5' at http://localhost:43287


In [22]:
print_qmodel_summary(new_model)

conv_0               f=16 quantized_bits(6,0,1,alpha=1.0) 
bn_conv_0            is normal keras bn layer
conv_act_0           quantized_relu(4,2)
conv_1               f=16 quantized_bits(6,0,1,alpha=1.0) 
bn_conv_1            is normal keras bn layer
conv_act_1           quantized_relu(4,2)
conv_2               f=24 quantized_bits(6,0,1,alpha=1.0) 
bn_conv_2            is normal keras bn layer
conv_act_2           quantized_relu(4,2)
dense_0              u=42 quantized_bits(2,0,1,alpha=1.0) 
bn_dense_0           is normal keras bn layer
dense_act_0          quantized_relu(8,2)
dense_1              u=64 quantized_bits(2,0,1,alpha=1.0) 
bn_dense_1           is normal keras bn layer
dense_act_1          quantized_relu(8,2)



In [23]:
import hls4ml
import plotting


hls_config_aq = hls4ml.utils.config_from_keras_model(new_model, granularity='name')
hls_config_aq['Model']['ReuseFactor'] = 8
hls_config_aq['Model']['Precision'] = 'ap_fixed<16,6>'
hls_config_aq['LayerName']['output_softmax']['Strategy'] = 'Stable'
plotting.print_dict(hls_config_aq)

cfg_aq = hls4ml.converters.create_config(backend='Vivado')
cfg_aq['IOType'] = 'io_stream'  # Must set this if using CNNs!
cfg_aq['HLSConfig'] = hls_config_aq
cfg_aq['KerasModel'] = new_model
cfg_aq['OutputDir'] = 'autoqkeras_cnn/'
cfg_aq['XilinxPart'] = 'xczu5ev-sfvc784-1-i'

hls_model_aq = hls4ml.converters.keras_to_hls(cfg_aq)
hls_model_aq.compile()


Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 28, 28, 1]], output shape: [None, 28, 28, 1]
Layer name: conv_0, layer type: QConv2D, input shapes: [[None, 28, 28, 1]], output shape: [None, 26, 26, 16]
Layer name: bn_conv_0, layer type: BatchNormalization, input shapes: [[None, 26, 26, 16]], output shape: [None, 26, 26, 16]
Layer name: conv_act_0, layer type: Activation, input shapes: [[None, 26, 26, 16]], output shape: [None, 26, 26, 16]
Layer name: pool_0, layer type: MaxPooling2D, input shapes: [[None, 26, 26, 16]], output shape: [None, 13, 13, 16]
Layer name: conv_1, layer type: QConv2D, input shapes: [[None, 13, 13, 16]], output shape: [None, 11, 11, 16]
Layer name: bn_conv_1, layer type: BatchNormalization, input shapes: [[None, 11, 11, 16]], output shape: [None, 11, 11, 16]
Layer name: conv_act_1, layer type: Activation, input shapes: [[None, 11, 11, 16]], output shape: [None, 11, 11, 16]
Layer name: pool_1, layer type: MaxPooling2

In [24]:
from sklearn.metrics import accuracy_score

y_predict_aq = aqmodel.predict(x_test)
y_predict_hls4ml_aq = hls_model_aq.predict(np.ascontiguousarray(x_test))


accuracy_keras = float(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_predict_aq, axis=1)))
accuracy_hls4ml = float(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_predict_hls4ml_aq, axis=1)))

print("Accuracy AutoQ Keras:  {}".format(accuracy_keras))
print("Accuracy AutoQ hls4ml: {}".format(accuracy_hls4ml))

313/313 [==============================] - 1s 2ms/step
Accuracy AutoQ Keras:  0.9615
Accuracy AutoQ hls4ml: 0.9627


In [ ]:
synth = True
if synth:
    hls_model_aq.build(csim=False, synth=True, vsynth=True)  
